In [1]:
import os, sys
sys.path.append(os.path.abspath("./Source"))

# Interactive User Guide

`rxprop` is all about creating reactive properties.

Reactive properties are properties that notify anyone who's interested whenever they change.

You make these properties using *decorators*, the same way you make ordinary class properties in Python.

There are two kinds of reactive properties: **value properties** and **computed properties**.

- **Value property**: A reactive property that stores a value. You can get it or set it just like an ordinary property, and it will notify watchers whenever you set it.

- **Computed property**: A reactive property that computes a result. It will watch for changes that would affect its value, and notify watchers when the result changes.

## Value Properties

A value property is a reactive property that simply stores a value.

You can get or set it just like an ordinary property, and it will notify watchers whenever it is set to a new value.

You create a value property using the `rx_value` decorator:

In [2]:
import rxprop as rx

class MyClass:
    
    @rx.value
    def my_value(self) -> int:
        return 0

In this example, `my_value` is a reactive value.

Unlike an ordinary Python `@property` which decorates a getter method, our `@rx.rx_value` decorates the default (initial) value of the property.

So if we get the value, we will get this initial value (zero):

In [3]:
a = MyClass()
print("my_value =", a.my_value)

my_value = 0


We can change the value like an ordinary property:

In [4]:
a.my_value = 1
print("my_value =", a.my_value)

my_value = 1


Now the magic part: we can watch for changes to the value using the `watch` function.

This function returns an async iterator that yields the value whenever it changes. So we need to make a small `consumer` method to process the notifications.

In [5]:
from asyncio import create_task, sleep

async def consumer():
    async for i in rx.watchp(a, MyClass.my_value):
        print("Value notification! my_value =", str(i))

consumer_task = create_task(consumer())
await sleep(0) # let the consumer start

a.my_value = 2
await sleep(0) # let the consumer catch the notification

Value notification! my_value = 1
Value notification! my_value = 2


We get two notifications. The first is the current value, which gets sent to the watcher as soon as it starts watching. The second is the new value we set.

Note that notifications only happen *once at the start*, then whenever the value *changes*. So if we set the value to 2 again, the watcher won't be notified.

In [6]:
a.my_value = 2
await sleep(0)

(We should clean up our consumer ready for the next section.)

In [7]:
consumer_task.cancel()

True

## Computed Properties

A computed property is a reactive property that computes a result.

It notifies watchers when that result changes.

In [8]:
import rxprop as rx

class MyClass:
    
    @rx.value
    def my_value(self) -> int:
        return 0
    
    @rx.computed
    def my_computed(self) -> int:
        return self.my_value * 2

Here, we've taken our class from before and added a second property, a computed property.

The `@rx_computed` decorator *does* go onto a getter function, like `@property`, and unlike `@rx_value`.

We can get its value, like a normal property.

In [9]:
a = MyClass()
print("a.my_value =", a.my_value)
print("a.my_computed =", a.my_computed)

a.my_value = 0
a.my_computed = 0


But we can't set its value, since it has no setter.

In [10]:
try:
    a.my_computed = 1
except Exception as e:
    print("Oops!", e)

As any property with a getter would do, `my_computed` reflects changes to `my_value` when we actively ask it.

In [11]:
a.my_value = 2
print("a.my_value =", a.my_value)
print("a.my_computed =", a.my_computed)

a.my_value = 2
a.my_computed = 4


But what about notifications? An ordinary `@property` wouldn't pass those through. That's what makes `my_computed` special.

When it computes a value, the `@rx.computed` property also keeps a look out for **dependencies**: other reactive properties that might send change notifications. It watches these dependencies for changes, and then notifies its own watchers in turn.

In [12]:
from asyncio import create_task, sleep

async def consumer():
    async for i in rx.watchp(a, MyClass.my_computed):
        print("Computed notification! my_computed =", str(i))

consumer_task = create_task(consumer())
await sleep(0.1) # let the consumer start

a.my_value = 3
await sleep(0.1) # let the consumer catch the notification

Computed notification! my_computed = 4
Computed notification! my_computed = 6


As in the example with `@rx.rx_value`, we get two notifications: one with the *current value*, and one with the new value after we changed `my_value`.

(Cleanup time!)

In [13]:
consumer_task.cancel()

True